In [246]:
import pandas as pd
import numpy as np
import re
import scipy.stats
import statsmodels.stats.multitest
import matplotlib.pyplot as plt
from matplotlib import pylab
import seaborn as sns
import cptac

In [334]:
CCDS = pd.read_csv('CCDS2Sequence.current.txt', sep = '\t')
og_colon = pd.read_csv('colon_somatic.maf.txt', sep = '\t')
og_colon

,Chr,Start,End,Ref,Alt,Variant_Type,Variant_Function,Gene,mRNA,mRNA_Change,Protein_Change,AA_Ref,AA_Pos,AA_Var,SampleID
0,19,58864357,58864357,A,G,nonsynonymous SNV,protein-altering,A1BG,NM_130786,T277C,C93R,C,93,R,05CO044
1,19,58864307,58864307,C,G,nonsynonymous SNV,protein-altering,A1BG,NM_130786,G327C,E109D,E,109,D,01CO005
2,19,58858758,58858758,C,T,nonsynonymous SNV,protein-altering,A1BG,NM_130786,G1441A,E481K,E,481,K,11CO070
3,19,58858859,58858859,C,T,nonsynonymous SNV,protein-altering,A1BG,NM_130786,G1340A,R447H,R,447,H,11CO059
4,19,58864354,58864354,G,A,nonsynonymous SNV,protein-altering,A1BG,NM_130786,C280T,R94C,R,94,C,05CO028
5,10,52595911,52595911,G,T,nonsynonymous SNV,protein-altering,A1CF,NM_014576,C527A,T176N,T,176,N,09CO013
6,10,52595911,52595911,G,T,nonsynonymous SNV,protein-altering,A1CF,NM_138932,C527A,T176N,T,176,N,09CO013
7,10,52595911,52595911,G,T,nonsynonymous SNV,protein-altering,A1CF,NM_001198818,C527A,T176N,T,176,N,09CO013
8,10,52595911,52595911,G,T,nonsynonymous SNV,protein-altering,A1CF,NM_138933,C551A,T184N,T,184,N,09CO013
9,10,52595911,52595911,G,T,nonsynonymous SNV,protein-altering,A1CF,NM_001198820,C551A,T184N,T,184,N,09CO013


In [332]:
mylist = og_colon['mRNA'].tolist()

In [170]:
columns = og_colon.columns.tolist()
columns.remove('mRNA')
columns.remove('Protein_Change')
for name in columns:
    og_colon = og_colon.drop(name, 1)

In [171]:
comp = CCDS['nucleotide_ID'].tolist()

new_comp = {}
for ID in comp:
    if ID[-2] == '.':
        ID = ID[:-2]
        new_comp[ID] = ID
    elif ID[-3] == '.':
        ID = ID[:-2]
        new_comp[ID] = ID
    else:
        new_comp[ID] = ID

,#ccds,original_member,current_member,source,nucleotide_ID,protein_ID,status_in_CCDS,sequence_status
0,CCDS2.2,1,0,NCBI,NM_152486.2,NP_689699.2,Updated,0
1,CCDS2.2,0,1,NCBI,NM_152486.3,NP_689699.2,Accepted,1
2,CCDS2.2,1,1,"EBI,WTSI",ENST00000342066.7,ENSP00000342313.3,Accepted,1
3,CCDS3.1,1,0,NCBI,NM_015658.3,NP_056473.2,Updated,0
4,CCDS3.1,0,1,NCBI,NM_015658.4,NP_056473.3,Accepted,1
5,CCDS3.1,1,1,"EBI,WTSI",ENST00000327044.6,ENSP00000317992.6,Accepted,1
6,CCDS4.1,1,0,NCBI,NM_032129.2,NP_115505.2,Updated,0
7,CCDS4.1,0,1,NCBI,NM_032129.3,NP_115505.2,Accepted,1
8,CCDS4.1,1,1,"EBI,WTSI",ENST00000379410.7,ENSP00000368720.3,Accepted,1
9,CCDS5.1,1,0,NCBI,NM_021170.3,NP_066993.1,Updated,0


In [204]:
NCBI_dict = {}
CCDS_dict = {}


for transcript in CCDS['nucleotide_ID']:
    
    name = CCDS.loc[CCDS['nucleotide_ID'] == transcript, '#ccds'].tolist()
    name = name[0]
    check = CCDS.loc[CCDS['nucleotide_ID'] == transcript, 'current_member'].tolist()
    check = check[0]
    source = CCDS.loc[CCDS['nucleotide_ID'] == transcript, 'source'].tolist()
    source = source[0]
    
    #Trimming the decimals from the transcript ID's
    ID = transcript
    if ID[-2] == '.':
        short_ID = ID[:-2]
    elif ID[-3] == '.':
        short_ID = ID[:-3]
    else:
        short_ID = ID
    
    #only using the currently accepts IDs for the transcripts
    if check == 1:
        if source == 'NCBI':
            NCBI_dict[short_ID] = name
        if source == 'EBI,WTSI':
            CCDS_dict[name] = short_ID
            
final_dict = {}

for key in NCBI_dict.keys():
    name = NCBI_dict[key]
    final_dict[key] = CCDS_dict[name]

In [133]:
new_colon = og_colon
for mutation in og_colon['mRNA']:
    if mutation in NCBI_dict:
        name = NCBI_dict[mutation]
        ID = CCDS_dict[name]
        new_colon.loc[og_colon['mRNA'] == mutation, 'mRNA'] = ID

In [134]:
new_colon1 = new_colon

In [135]:
for row in new_colon.iterrows():
    info = row[1]
    ID = row[0]
    mut = info[1]
    new_mut = 'p.'+ str(mut)
    
    new_colon1.loc[new_colon1.index == ID, 'Protein_Change'] = new_mut

In [156]:
num = 0
for transcript_id in new_colon1['mRNA']:
    if transcript_id[0] == 'N':
        new_colon1.drop(new_colon1['mRNA'] == transcript_id, axis = 0)

KeyError: '[False False False ... False False False] not found in axis'

In [ ]:
new_colon1.to_csv('colon_mutations', sep = '\t', index = False)

In [153]:
num

4583

In [207]:
co = cptac.Colon()
d1 = co.get_mutations()
d1.head()

,Gene,Mutation,Location
Sample_ID,,,
S001,ABCA8,nonsynonymous SNV,F139Y
S001,ABCA8,nonsynonymous SNV,F139Y
S001,ABCA8,nonsynonymous SNV,F139Y
S001,ACP7,nonsynonymous SNV,R188W
S001,ADAMTS16,nonsynonymous SNV,D299N


In [213]:
index_list = new_colon1.index.tolist()

In [235]:
index_list = og_colon.index.tolist()
for index in index_list:
    if index in new_comp:
        new = new_colon1.loc[new_colon1.index == index, 'Protein_Change'].tolist()
        new = new[0]
        test_colon.loc[test_colon.index == index, 'Protein_Change'] = new
    else:
        test_colon = test_colon.drop(index = index, axis = 0)
    

In [219]:
test = new_colon1.loc[new_colon1.index == 21, 'mRNA'].tolist()

In [230]:
new_comp = {}
new_index = new_colon1.index.tolist()

for index in new_index:
    new_comp[index] = index

In [287]:
test_colon.to_csv('colon_mutations_5.maf', sep = '\t', index = False)

In [281]:
things = test_colon['mRNA'].tolist()

In [ ]:
for thing in things:
    if thing[-2] == '.':
        short = thing[:-2]
        test_colon
    

In [282]:
test_colon = pd.read_csv('colon_mutations.maf', sep = '\t')

In [253]:
test_colon['mRNA'] = test_colon['mRNA'].str.replace(r'\.\d{1,2}$', '')

In [258]:
test_colon.drop_duplicates(inplace = True)

In [267]:
test_colon['Transcript_ID'] = test_colon['mRNA']
test_colon['HGVSp_Short'] = test_colon['Protein_Change']

In [283]:
output_colon = test_colon.head(50)


In [277]:
output_colon_2 = og_colon.head(50)


In [275]:
output_colon.to_csv('changed_colon_mutations', sep = '\t')
output_colon_2.to_csv('original_colon_mutations', sep = '\t')

,mRNA,Protein_Change
0,ENST00000263100.7,p.C93R
1,ENST00000263100.7,p.E109D
2,ENST00000263100.7,p.E481K
3,ENST00000263100.7,p.R447H
4,ENST00000263100.7,p.R94C
5,ENST00000374001.6,p.T176N
6,ENST00000373993.5,p.T176N
7,ENST00000374001.6,p.T176N
8,ENST00000395495.5,p.T184N
9,ENST00000395495.5,p.T184N


In [285]:
test_colon = test_colon.rename(columns={'Chr': 'Chromosome', 'Start': 'Start_Position', 'End': 'End_Position', 'Ref': 'Reference_Allele', 'Alt': 'Tumor_Seq_Allele2', 'SampleID': 'Tumor_Sample_Barcode'})

In [280]:
maf_columns = ('Hugo_Symbol', 'Entrez_Gene_Id', 'Center', 'NCBI_Build', 'Chromosome', 'Start_Position','End_Position','Strand','Variant_Classification','Variant_Type','Reference_Allele','Tumor_Seq_Allele1','Tumor_Seq_Allele2','dbSNP_RS','dbSNP_Val_Status','Tumor_Sample_Barcode','Matched_Norm_Sample_Barcode','Match_Norm_Seq_Allele1','Match_Norm_Seq_Allele2','Tumor_Validation_Allele1','Tumor_Validation_Allele2','Match_Norm_Validation_Allele1','Match_Norm_Validation_Allele2','Verification_Status','Validation_Status','Mutation_Status','Sequencing_Phase','Sequence_Source','Validation_Method','Score','BAM_File','Sequencer','Tumor_Sample_UUID','Matched_Norm_Sample_UUID','HGVSc','HGVSp','HGVSp_Short','Transcript_ID','Exon_Number','t_depth','t_ref_count','t_alt_count','n_depth','n_ref_count','n_alt_count','callers','all_effects','Allele','Gene','Feature','Feature_type','Consequence','cDNA_position','CDS_position','Protein_position','Amino_acids','Codons','Existing_variation','ALLELE_NUM','DISTANCE','STRAND_VEP','SYMBOL','SYMBOL_SOURCE','HGNC_ID','BIOTYPE','CANONICAL','CCDS','ENSP','SWISSPROT','TREMBL','UNIPARC','RefSeq','SIFT','PolyPhen','EXON','INTRON','DOMAINS','GMAF','AFR_MAF','AMR_MAF','ASN_MAF','EAS_MAF','EUR_MAF','SAS_MAF','AA_MAF','EA_MAF','CLIN_SIG','SOMATIC','PUBMED','MOTIF_NAME','MOTIF_POS','HIGH_INF_POS','MOTIF_SCORE_CHANGE','IMPACT','PICK','VARIANT_CLASS','TSL','HGVS_OFFSET','PHENO','MINIMISED','ExAC_AF','ExAC_AF_AFR','ExAC_AF_AMR','ExAC_AF_EAS','ExAC_AF_FIN','ExAC_AF_NFE','ExAC_AF_OTH','ExAC_AF_SAS','GENE_PHENO','FILTER','flanking_bps','variant_id','variant_qual','ExAC_AF_Adj','ExAC_AC_AN_Adj','ExAC_AC_AN','ExAC_AC_AN_AFR','ExAC_AC_AN_AMR','ExAC_AC_AN_EAS','ExAC_AC_AN_FIN','ExAC_AC_AN_NFE','ExAC_AC_AN_OTH','ExAC_AC_AN_SAS','ExAC_FILTER')

In [288]:
og_colon

,Chr,Start,End,Ref,Alt,Variant_Type,Variant_Function,Gene,mRNA,mRNA_Change,Protein_Change,AA_Ref,AA_Pos,AA_Var,SampleID
0,19,58864357,58864357,A,G,nonsynonymous SNV,protein-altering,A1BG,NM_130786,T277C,C93R,C,93,R,05CO044
1,19,58864307,58864307,C,G,nonsynonymous SNV,protein-altering,A1BG,NM_130786,G327C,E109D,E,109,D,01CO005
2,19,58858758,58858758,C,T,nonsynonymous SNV,protein-altering,A1BG,NM_130786,G1441A,E481K,E,481,K,11CO070
3,19,58858859,58858859,C,T,nonsynonymous SNV,protein-altering,A1BG,NM_130786,G1340A,R447H,R,447,H,11CO059
4,19,58864354,58864354,G,A,nonsynonymous SNV,protein-altering,A1BG,NM_130786,C280T,R94C,R,94,C,05CO028
5,10,52595911,52595911,G,T,nonsynonymous SNV,protein-altering,A1CF,NM_014576,C527A,T176N,T,176,N,09CO013
6,10,52595911,52595911,G,T,nonsynonymous SNV,protein-altering,A1CF,NM_138932,C527A,T176N,T,176,N,09CO013
7,10,52595911,52595911,G,T,nonsynonymous SNV,protein-altering,A1CF,NM_001198818,C527A,T176N,T,176,N,09CO013
8,10,52595911,52595911,G,T,nonsynonymous SNV,protein-altering,A1CF,NM_138933,C551A,T184N,T,184,N,09CO013
9,10,52595911,52595911,G,T,nonsynonymous SNV,protein-altering,A1CF,NM_001198820,C551A,T184N,T,184,N,09CO013


In [295]:
example = pd.read_csv('example.maf', '\t')
test_colon

,Chromosome,Start_Position,End_Position,Reference_Allele,Tumor_Seq_Allele2,Variant_Type,Variant_Function,Gene,mRNA,mRNA_Change,Protein_Change,AA_Ref,AA_Pos,AA_Var,Tumor_Sample_Barcode
0,19,58864357,58864357,A,G,nonsynonymous SNV,protein-altering,A1BG,ENST00000263100.7,T277C,p.C93R,C,93,R,05CO044
1,19,58864307,58864307,C,G,nonsynonymous SNV,protein-altering,A1BG,ENST00000263100.7,G327C,p.E109D,E,109,D,01CO005
2,19,58858758,58858758,C,T,nonsynonymous SNV,protein-altering,A1BG,ENST00000263100.7,G1441A,p.E481K,E,481,K,11CO070
3,19,58858859,58858859,C,T,nonsynonymous SNV,protein-altering,A1BG,ENST00000263100.7,G1340A,p.R447H,R,447,H,11CO059
4,19,58864354,58864354,G,A,nonsynonymous SNV,protein-altering,A1BG,ENST00000263100.7,C280T,p.R94C,R,94,C,05CO028
5,10,52595911,52595911,G,T,nonsynonymous SNV,protein-altering,A1CF,ENST00000374001.6,C527A,p.T176N,T,176,N,09CO013
6,10,52595911,52595911,G,T,nonsynonymous SNV,protein-altering,A1CF,ENST00000373993.5,C527A,p.T176N,T,176,N,09CO013
7,10,52595911,52595911,G,T,nonsynonymous SNV,protein-altering,A1CF,ENST00000374001.6,C527A,p.T176N,T,176,N,09CO013
8,10,52595911,52595911,G,T,nonsynonymous SNV,protein-altering,A1CF,ENST00000395495.5,C551A,p.T184N,T,184,N,09CO013
9,10,52595911,52595911,G,T,nonsynonymous SNV,protein-altering,A1CF,ENST00000395495.5,C551A,p.T184N,T,184,N,09CO013


In [326]:
example.head()


,Hugo_Symbol,Chromosome,Start_Position,End_Position,Variant_Classification,Reference_Allele,Tumor_Seq_Allele1,Tumor_Seq_Allele2,Tumor_Sample_Barcode,Transcript_ID,HGVSp_Short
0,EGFR,7,55233109,55233109,Missense_Mutation,G,G,A,TCGA-02-0003-01A-01D-1490-08,ENST00000275493,p.C620Y
1,FGFR2,10,123298220,123298220,Missense_Mutation,G,G,T,TCGA-02-0033-01A-01D-1490-08,ENST00000457416,p.Q212K
2,EGFR,7,55221822,55221822,Missense_Mutation,C,C,A,TCGA-02-2485-01A-01D-1494-08,ENST00000275493,p.A289D
3,PDGFRA,4,55133510,55133510,Missense_Mutation,G,G,A,TCGA-04-1341-01A-01W-0486-08,ENST00000257290,p.V272M
4,ERBB2,17,37883786,37883786,Missense_Mutation,G,G,C,TCGA-04-1342-01A-01W-0486-08,ENST00000269571,p.C1133S


In [331]:
final_colon.head()

,Hugo_Symbol,Chromosome,Start_Position,End_Position,Variant_Classification,Reference_Allele,Tumor_Seq_Allele1,Tumor_Seq_Allele2,Tumor_Sample_Barcode,Transcript_ID,HGVSp_Short
0,A1BG,19,58864357,58864357,protein-altering,A,A,G,05CO044,ENST00000263100,p.C93R
1,A1BG,19,58864307,58864307,protein-altering,C,C,G,01CO005,ENST00000263100,p.E109D
2,A1BG,19,58858758,58858758,protein-altering,C,C,T,11CO070,ENST00000263100,p.E481K
3,A1BG,19,58858859,58858859,protein-altering,C,C,T,11CO059,ENST00000263100,p.R447H
4,A1BG,19,58864354,58864354,protein-altering,G,G,A,05CO028,ENST00000263100,p.R94C


In [328]:
final_colon = pd.DataFrame()
final_colon['Hugo_Symbol'] = test_colon['Gene']
final_colon['Chromosome'] = test_colon['Chromosome']
final_colon['Start_Position'] = test_colon['Start_Position']
final_colon['End_Position'] = test_colon['End_Position']
final_colon['Variant_Classification'] = test_colon['Variant_Function']
final_colon['Reference_Allele'] = test_colon['Reference_Allele']
final_colon['Tumor_Seq_Allele1'] = test_colon['Reference_Allele']
final_colon['Tumor_Seq_Allele2'] = test_colon['Tumor_Seq_Allele2']
final_colon['Tumor_Sample_Barcode'] = test_colon['Tumor_Sample_Barcode']
final_colon['Transcript_ID'] = test_colon['mRNA']
final_colon['HGVSp_Short'] = test_colon['Protein_Change']

In [329]:
final_colon['Transcript_ID'] = final_colon['Transcript_ID'].str.replace(r'\.\d{1,2}$', '')

In [330]:
final_colon.head()

,Hugo_Symbol,Chromosome,Start_Position,End_Position,Variant_Classification,Reference_Allele,Tumor_Seq_Allele1,Tumor_Seq_Allele2,Tumor_Sample_Barcode,Transcript_ID,HGVSp_Short
0,A1BG,19,58864357,58864357,protein-altering,A,A,G,05CO044,ENST00000263100,p.C93R
1,A1BG,19,58864307,58864307,protein-altering,C,C,G,01CO005,ENST00000263100,p.E109D
2,A1BG,19,58858758,58858758,protein-altering,C,C,T,11CO070,ENST00000263100,p.E481K
3,A1BG,19,58858859,58858859,protein-altering,C,C,T,11CO059,ENST00000263100,p.R447H
4,A1BG,19,58864354,58864354,protein-altering,G,G,A,05CO028,ENST00000263100,p.R94C


In [325]:
final_colon.to_csv('colon_mutations_7.maf', sep = '\t', index = False)

In [327]:
final_colon.head(50).to_csv('reformatted_colon.maf', sep = '\t', index = False)